# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd 

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
pprint(mongo.list_uk_food())

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collections = db.list_establishments()
pprint(collections)

In [ ]:
# review the collections in our new database
collections = db.list_establishments()
pprint(collections)

In [ ]:
# review a document in the establishments collection
db = mongo.establishments 
collection = db.establishments 
document = collection.find_one()
pprint(document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {"name": "Penang Flavours",
"location": "Greenwich",
"rating": None,
"cuisine": "Halal",
"status": "unrated"}

In [ ]:
# Insert the new restaurant into the collection
db = mongo.uk_food 
collection = db.establishments

In [ ]:
# Check that the new restaurant was inserted
inserted_restaurant = collection.find_one({"name": "Penang Flavours"})
if inserted_restaurant: 
    print("New restaurant data:")   
    pprint(inserted_restaurant)
else:
    print("New restaurant not found in the collection")  

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
db = mongo.uk_food 
collection = db.establishments
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
result = collection.find_one(query, projection)
if result:
    print("BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':")
    pprint(result)
else:
    print("Document not found.")

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
update_result = collection.update_one(query, {"$set": new_restaurant_data})
if update_result.modified_count > 0: 
    print("Update completed successfully")
else: 
    print("Update failed")    

In [ ]:
# Confirm that the new restaurant was updated
db = mongo.uk_food 
collection = db.establishments
query = {"name": "Penang Flavours"}
updated_restaurant = collection.find_one(query)
if updated_restaurant:
    print("Updated restaurant data:")
    pprint(updated_restaurant)
else:
    print("Updated restaurant not found in the collection.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
db = mongo.uk_food 
collection = db.establishments
query = {"LocalAuthorityName": "Dover"}
count = collection.count_documents(query)
print("Number of documents with LocalAuthorityName 'Dover':", count)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
delete_result = collection.delete_many(query)
if delete_result.deleted_count > 0:
    print("Deletion completed successfully. Deleted documents count:", delete_result.deleted_count)
else:
    print("No documents found to delete with LocalAuthorityName 'Dover'.")

In [ ]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
result = collection.find_one(query)
if result:
    print("Documents with LocalAuthorityName 'Dover' still exist.")
else:
    print("No documents found with LocalAuthorityName 'Dover'.")

In [ ]:
# Check that other documents remain with 'find_one'
result = collection.find_one()
if result:
    print("At least one document remains in the collection.")
else:
    print("No documents found in the collection.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
query = {"longitude": {"$type": "string"}, "latitude": {"$type": "string"}}
update = {"$set": {"longitude": {"$toDouble": "$longitude"}, "latitude": {"$toDouble": "$latitude"}}}
update_result = collection.update_many(query, update)
if update_result.modified_count > 0:
    print("Update completed successfully. Number of documents updated:", update_result.modified_count)
else:
    print("No documents found to update.")


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
query = {"RatingValue": {"$type": "string"}}
update = [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
update_result = collection.update_many(query, update)
if update_result.modified_count > 0:
    print("Update completed successfully. Number of documents updated:", update_result.modified_count)
else:
    print("No documents found to update.")

In [ ]:
# Check that the coordinates and rating value are now numbers
result = collection.find_one()
if result and isinstance(result["latitude"], float) and isinstance(result["longitude"], float) and isinstance(result["RatingValue"], int):
    print("Coordinates and RatingValue are now numbers.")
else:
    print("Error: Coordinates and/or RatingValue are not numbers.")